# AtlanticWave SDX Cross-Domain Orchestration and Provisioning for Facilities and R&E Networks user interfaces with the SDXLib in Fabric Jupyter Notebook





# FABRIC users authentication using CILogon

This Jupyter notebook will walk you through implementing authentication using Fabric CILogon to pass JSON authentication to SDX project. The "FABRIC users authentication using CILogon" experiment tests the configuration of your environment to ensure you can create and access resources on FABRIC.  Specifically, the experiment deploys a slice of resources with a single virtual machine and confirms you can log into the virtual machine using FABLib methods.

## Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).


## Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,1ecd9d6a-7701-40fa-b78e-b2293c9526ed
Bastion Host,bastion-1.fabric-testbed.net
Bastion Username,lmarinve_0000011754
Bastion Private Key File,/home/fabric/work/fabric_config/newonekey
Slice Public Key File,/home/fabric/work/fabric_config/silverkey.pub
Slice Private Key File,/home/fabric/work/fabric_config/silverkey


## Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [2]:
slice_name = "Slice-AWSDX"

facility_port_site='FIU'

### Facility port info

#### Find the facility ports on the site
List the facility ports
- Facility Port Name
- Available VLAN range
- Currently allocated VLAN range

In [3]:
cell_output = 'pandas'
output_list = fablib.list_facility_ports(filter_function=lambda x: x['site_name'] == facility_port_site, output=cell_output)

Name,Site,Interface Name,VLAN Range,Allocated VLAN Range,Local Name,Device Name,Region
AmLight-EXP-Layer2-FIU,FIU,port+fiu-data-sw:HundredGigE0/0/0/23:facility+AmLight-EXP-Layer2-FIU-int,['4000-4019'],['4015'],HundredGigE0/0/0/23,N/A,N/A
AmLight-Layer3-FIU,FIU,port+fiu-data-sw:HundredGigE0/0/0/23:facility+AmLight-Layer3-FIU-int,['3001-3008'],N/A,HundredGigE0/0/0/23,N/A,N/A


#### Select the Facility Port on the site

In [4]:
facility_port_name = 'AmLight-EXP-Layer2-FIU'

facility_port_vlan='4015'

### Create Slice

In [5]:
slice = fablib.new_slice(name=slice_name)

# Example: One VM on FABRIC-STAR with a NIC - sharedNIC (basic) or dedicated smartNIC (ConnectX_6)
node = slice.add_node(name=f"Node1", site='STAR')

node_iface = node.add_component(model='NIC_Basic', name="nic1").get_interfaces()[0]

# Add facility port to the network service configuration along with the node interface(s)

In [6]:
facility_port = slice.add_facility_port(name=facility_port_name, site=facility_port_site, vlan=facility_port_vlan)
facility_port_interface =facility_port.get_interfaces()[0]

net = slice.add_l2network(name=f'net_facility_port', interfaces=[])
net.add_interface(node_iface)
net.add_interface(facility_port_interface)

# Print Facility port Info

In [7]:
print(f"facility_port.get_site(): {facility_port.get_site()}")
print(f"facility_port.get_fim_interface(): {facility_port.get_fim_interface()}")

facility_port.get_site(): FIU
facility_port.get_fim_interface(): {'node_id': '7df901b0-b59e-4596-b1ec-73092f754528', 'name': 'AmLight-EXP-Layer2-FIU', 'site': 'FIU', 'type': 'Facility'}


## Observe the Slice's Attributes


In [8]:
slice = fablib.get_slice(slice_name)

In [9]:
slice.show()
slice.list_nodes()
slice.list_networks()
slice.list_interfaces()


ID,9cdbea32-cb32-4c3e-b810-1270df1fec42
Name,Slice-AWSDX
Lease Expiration (UTC),2025-03-06 05:09:00 +0000
Lease Start (UTC),2025-02-26 05:09:00 +0000
Project ID,1ecd9d6a-7701-40fa-b78e-b2293c9526ed
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
45e61a7a-a838-4726-ade0-4218e01d3f60,Node1,2,8,10,default_rocky_8,qcow2,star-w3.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:fe1c:7d3d,Active,,ssh -i /home/fabric/work/fabric_config/silverkey -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:fe1c:7d3d,/home/fabric/work/fabric_config/silverkey.pub,/home/fabric/work/fabric_config/silverkey


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
0ed0c353-9791-426f-8aba-df7aba825e36,net_facility_port,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,1E:64:D4:3A:C4:B0,eth1,eth1,192.168.1.101,4,HundredGigE0/0/0/9


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,1E:64:D4:3A:C4:B0,eth1,eth1,192.168.1.101,4,HundredGigE0/0/0/9


## Run the Experiment

In [10]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Configure the interface(s) of the VM(s) with designated subnet
subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[2:]

In [11]:
node1 = slice.get_node(name=f"Node1")        
node1_iface = node1.get_interface(network_name=f'net_facility_port') 
node1_addr = available_ips.pop(99)
print(f"node1_addr: {node1_addr}")
node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)

stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_physical_os_interface_name()} up')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_os_interface()} up')

node1_addr: 192.168.1.101
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 1e:64:d4:3a:c4:b0 brd ff:ff:ff:ff:ff:ff
    altname enp7s0
    inet 192.168.1.101/24 scope global eth1
       valid_lft forever preferred_lft forever


In [12]:
# Ping the node(s) that are active on the Facility Port side (e.g 192.168.1.10)
node1 = slice.get_node(name=f"Node1")     
node1_iface = node1.get_interface(network_name=f'net_facility_port') 

stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')   # This is expected to fail. 

PING 192.168.1.10 (192.168.1.10) 56(84) bytes of data.
From 192.168.1.101 icmp_seq=1 Destination Host Unreachable
From 192.168.1.101 icmp_seq=2 Destination Host Unreachable
From 192.168.1.101 icmp_seq=3 Destination Host Unreachable
From 192.168.1.101 icmp_seq=4 Destination Host Unreachable
From 192.168.1.101 icmp_seq=5 Destination Host Unreachable

--- 192.168.1.10 ping statistics ---
5 packets transmitted, 0 received, +5 errors, 100% packet loss, time 4075ms
pipe 3


# Import SDXLIB and FABLib Libraries

In [13]:
from pprint import pprint
from sdxlib.sdx_token_auth import TokenAuthentication
from sdxlib.sdx_client import SDXClient
from sdxlib.sdx_exception import SDXException

# Instantiate the TokenAuthentication class with the correct token file path (if needed)

In [14]:
# Read token from Jupyter Hub environment
import os

# token_file_path = "expired_id_token.json"
token_file_path = os.getenv("FABRIC_TOKEN_LOCATION", "/home/fabric/.tokens.json")

# Specify proxy_host
proxy_hostname = "sdxapi.atlanticwave-sdx.ai"

# Specify proxy_port
proxy_port = "443"

# Specify the endpoint
endpoint = "topology"  # Replace with your desired endpoint

# Instantiate the TokenAuthentication class with the correct token file path and endpoint
token_auth = TokenAuthentication(
    token_path=token_file_path,
    proxy_hostname=proxy_hostname,
    proxy_port=proxy_port,
    endpoint=endpoint)

In [15]:
# Load the token
token_auth.load_token()

# Print the decoded token for debugging (optional)
pprint(token_auth.token_decoded)

# Validate the token
pprint(token_auth.validate_token())

fabric_token = token_auth.fabric_token

FABRIC Token Path: /home/fabric/.tokens.json
FABRIC JWT Token: eyJhbGciOiJSUzI1NiIsImtpZCI6ImI0MTUxNjcyMTExOTFlMmUwNWIyMmI1NGIxZDNiNzY2N2U3NjRhNzQ3NzIyMTg1ZTcyMmU1MmUxNDZmZTQzYWEiLCJ0eXAiOiJKV1QifQ.eyJzdWIiOiJodHRwOi8vY2lsb2dvbi5vcmcvc2VydmVyRS91c2Vycy83MjQzNiIsImlkcF9uYW1lIjoiRmxvcmlkYSBJbnRlcm5hdGlvbmFsIFVuaXZlcnNpdHkiLCJlcHBuIjoibG1hcmludmVAZml1LmVkdSIsImlzcyI6Imh0dHBzOi8vY2lsb2dvbi5vcmciLCJhY3IiOiJ1cm46b2FzaXM6bmFtZXM6dGM6U0FNTDoyLjA6YWM6Y2xhc3NlczpQYXNzd29yZFByb3RlY3RlZFRyYW5zcG9ydCIsImF1ZCI6ImNpbG9nb246L2NsaWVudF9pZC8yOTdkZGZhZGM1ZWFiYTVlYmE5Yzg5ODY4YWE5NjI3ZCIsIm5iZiI6MTc0MDk3NTc4MywiaWRwIjoiaHR0cHM6Ly9zaWdub24uZml1LmVkdS9pZHAiLCJhdXRoX3RpbWUiOjE3NDA5NjMyNDksImV4cCI6MTc0MDk5MDE4NCwiaWF0IjoxNzQwOTc1Nzg0LCJlbWFpbCI6ImxtYXJpbnZlQGZpdS5lZHUiLCJqdGkiOiJodHRwczovL2NpbG9nb24ub3JnL29hdXRoMi9pZFRva2VuLzdjNDMwMmEzNDhmMjMzZWUwNWY4NTdhODAwNzQxZjgyLzE3NDA5Njk2NTE5ODYiLCJwcm9qZWN0cyI6W3sibmFtZSI6IkF0bGFudGljV2F2ZS1TRFgiLCJ1dWlkIjoiMWVjZDlkNmEtNzcwMS00MGZhLWI3OGUtYjIyOTNjOTUyNmVkIiwidGFncyI6Wy

## Show the Slice's Attributes

## Configure the Subnet

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Configure the interface(s) of the VM(s) with designated subnet
subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[2:]

## Configure the Node

In [ ]:
node1 = slice.get_node(name=f"Node1")        
node1_iface = node1.get_interface(network_name=f'net_facility_port') 
node1_addr = available_ips.pop(99)
print(f"node1_addr: {node1_addr}")
node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)

stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_physical_os_interface_name()} up')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_os_interface()} up')

## List Nodes

In [ ]:
slice.list_nodes()


## List Networks

In [ ]:
slice.list_networks()


## List Interfaces

In [ ]:
slice.list_interfaces()

## Ping a node on the Amlight Facility Port side

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')   

## Assign SDX Controller URL

In [ ]:
# url = "http://controller.atlanticwave-sdx.net:8080/SDX-Controller"
url = "https://sdxapi.atlanticwave-sdx.ai"

## List SDX Domain Nodes and Ports


In [ ]:
client=SDXClient(fabric_token, url)
client.get_available_ports()

## Create SDX Layer 2 VPN

The below specification is the minimum information needed to create the layer 2 VPN. 

In [ ]:
client_name = "Test SDX SC24 L2VPN"
client_endpoints = [
    {"port_id": "urn:sdx:port:amlight.net:MIA-MI1-SW17:7", "vlan": "4015"}, 
    {"port_id": "urn:sdx:port:amlight.net:MIA-MI1-SW15:9", "vlan": "4015"},
]
client = SDXClient(fabric_token, url, client_name, client_endpoints)

In [ ]:
try:
    response = client.create_l2vpn()  
    print("L2VPN creation successful!")
    pprint(response)
except SDXException as e:
    print(f"L2VPN creation failed: {e}") 

## List All SDX L2VPNs

Both of the list methods have the option of query results being listed as a DataFrame(default) or as a JSON by passing the optional argument format='json'.

In [ ]:
client.get_all_l2vpns()

## Show created L2VPN Details

In [ ]:
client.get_l2vpn(service_id = response['service_id'])

## Ping a node on the Amlight Facility Port side

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')   

## Delete created SDX L2VPN

Please make certain to delete when you are finished. 

In [ ]:
client.delete_l2vpn(service_id = response['service_id'])

## Ping a node on the Amlight Facility Port side

In [ ]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')   